In [28]:

import sys
import string
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import cross_validation  
from sklearn.cross_validation import KFold
from sklearn.model_selection import cross_val_predict 
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report
from sklearn.neighbors import NearestNeighbors

%matplotlib inline

In [16]:
path='scores_1.xlsx'
train=pd.read_excel(path, sheetname='Matrix with data')
test=pd.read_excel(path, sheetname='test data')

C:\Users\sandhya\Anaconda3\lib\site-packages\pandas\util\_decorators.py:118: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)


In [17]:
train.columns.values

array(['sku', 'Feature 1', 'Feature 2', 'Feature 3', 'Feature 4',
       'Feature 5', 'Feature 6', 'Feature 7', 'Feature 8', 'Feature 9',
       'Feature 10', 'Feature 11', 'Feature 12', 'Feature 13',
       'Feature 14', 'Feature 15', 'Feature 16', 'Feature 17',
       'Feature 18', 'Feature 19', 'Feature 20', 'Feature 21',
       'Feature 22', 'Feature 23', 'Feature 24', 'Feature 25',
       'Feature 26', 'Feature 27', 'Feature 28', 'Feature 29',
       'Feature 30', 'Feature 31', 'Feature 32', 'Feature 33',
       'Feature 34', 'Feature 35', 'Feature 36', 'Feature 37',
       'Feature 38', 'Feature 39', 'Feature 40', 'Feature 41',
       'Feature 42', 'Feature 43', 'Feature 44', 'Feature 45',
       'Feature 46', 'Feature 47', 'Feature 48', 'Feature 49',
       'Feature 50', 'Feature 51', 'Feature 52', 'Feature 53',
       'Feature 54', 'Feature 55', 'Feature 56', 'Feature 57',
       'Feature 58', 'Feature 59', 'Feature 60', 'Feature 61',
       'Feature 62', 'Feature 63', 'Feature

In [18]:
train_x = train.iloc[:, 1:]
train_y = train.iloc[:,0]
test_x = test.iloc[:, 1:]
test_y = test.iloc[:,0]
test_y


0     Prod 1
1    Prod 41
2    Prod 16
Name: sku, dtype: object

In [19]:
test.head()

,sku,Feature 1,Feature 2,Feature 3,Feature 4,Feature 5,Feature 6,Feature 7,Feature 8,Feature 9,...,Feature 93,Feature 94,Feature 95,Feature 96,Feature 97,Feature 98,Feature 99,Feature 100,Feature 101,Feature 102
0,Prod 1,1.7,1.3,4.0,1.8,4.0,1.5,4.1,4.0,2.2,...,2.3,2.3,1.9,1.8,1.7,2.1,2.2,2.3,1.8,1.9
1,Prod 41,1.5,2.0,1.7,1.5,1.8,1.9,3.0,2.1,2.2,...,2.3,2.3,1.9,1.8,1.7,2.1,2.2,2.3,2.1,2.1
2,Prod 16,1.7,1.3,1.5,1.9,1.7,1.6,2.2,2.1,2.2,...,2.3,2.3,1.9,1.8,1.7,2.1,2.2,2.3,2.1,1.9


In [20]:
train.shape

(75, 103)

In [21]:
test.shape

(3, 103)

In [22]:
#cheching if there are any missing values
train.isnull().sum().max()

0

In [23]:
knn = KNeighborsClassifier(n_neighbors=3, algorithm='auto')

# fitting the model
knn.fit(train_x, train_y)

# predict the response
pred = knn.predict(test_x)

# evaluate accuracy
print(accuracy_score(test_y, pred))
print(pred)
print(test_y)
knn.kneighbors(test_x, 3, False)

0.333333333333
[' Prod 1' 'Prod 40' 'Prod 12']
0     Prod 1
1    Prod 41
2    Prod 16
Name: sku, dtype: object


array([[ 0,  1,  2],
       [39, 40, 41],
       [16, 11, 22]], dtype=int64)

In [24]:

precision_recall_fscore_support(test_y, pred, average='weighted')

C:\Users\sandhya\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\sandhya\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


(0.33333333333333331, 0.33333333333333331, 0.33333333333333331, None)

In [31]:
#This is a multi class classification. where each row is a class. So to find the accuracy of the model, I randomly,
#sampled 10 observations for testing.


cv_scores=[]
def val():
    """to find the cross validation score"""
    knn = KNeighborsClassifier(n_neighbors=1)
    knn.fit(train_x, train_y)
    for n in range(1):
        sampled_test= train.sample(n=10)
        sampled_test_x=sampled_test.iloc[:, 1:]
        sampled_test_y = sampled_test.iloc[:,0]
        pred_s = knn.predict(sampled_test_x)
        scores=accuracy_score(sampled_test_y, pred_s)
        cv_scores.append(scores.mean())
        
    print("%.2f %%\n" % (np.mean(cv_scores)*100))
    return
 
val()

100.00 %



In [30]:
def nearneighbours(k):
    """ Function to find the nearest neighbors"""
    nbrs = NearestNeighbors(n_neighbors=k, algorithm='auto').fit(train_x,train_y)

# Let's find the k-neighbors of each point in object X. To do that we call the kneighbors() function on object X.
    distances, indices = nbrs.kneighbors(test_x)
    yy=pd.DataFrame.from_records(indices)
    for i in range(len(test)):
        print('\n'+'Test case : '+str(i))
        print('Actual product taken as test case:   '+test.iloc[i,0]+'\n'+'\n'+'Products to be recommended:')
        print(train.iloc[yy.iloc[i],0])
    return



In [29]:
if __name__ == '__main__':
    nearneighbours(3)


Test case : 0
Actual product taken as test case:    Prod 1

Products to be recommended:
0     Prod 1
1     Prod 2
2     Prod 3
Name: sku, dtype: object

Test case : 1
Actual product taken as test case:   Prod 41

Products to be recommended:
39    Prod 40
40    Prod 41
41    Prod 42
Name: sku, dtype: object

Test case : 2
Actual product taken as test case:   Prod 16

Products to be recommended:
16    Prod 17
11    Prod 12
22    Prod 23
Name: sku, dtype: object
